This exercise will build off the previous module, where we connected to a Logica FHIR server and made a simple query for a specific patient:

```{}
# Setup
from fhir_pyrate import Pirate
import pandas as pd
pd.options.display.max_columns = None  # so we can display all columns

# Instantiating search object
search = Pirate(
    auth=None,
    base_url="https://api.logicahealth.org/FHIRResearchSynthea/open/"
)

# Query FHIR Data
patient_all = search.steal_bundles_to_dataframe(  
    resource_type="Patient",
    request_params={
        "_id": "11149"
    },
)
patient_all
```

## A Closer Look at JSON -\> DataFrame Conversion

Before cleaning up our dataframe, it is worth understanding exactly how `fhir_pyrate` is converting JSON data into a tabular format so we can know what the returned `pandas` DataFrame represents.

### General Example

JSON data is organized like a JavaScript object, meaning it is a collection of key-value pairs. The table below describes three major types of key-value pairs you will see in a FHIR bundle and how `fhir_pyrate` converts them into a tabular format.

+---------------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| Key-Value Pair Type | Description                                                                                      | `fhir_pyrate`'s tabular conversion                                                                                                          |
+=====================+==================================================================================================+=============================================================================================================================================+
| Simple              | Key is paired to a single value                                                                  | Key -\> column title                                                                                                                        |
|                     |                                                                                                  |                                                                                                                                             |
|                     |                                                                                                  | Value -\> row entry                                                                                                                         |
+---------------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| Nested              | Key is paired to a single object that contains multiple nested key-value pairs                   | Each nested key-value pair is a column/row (same as above) with the parent key string appended to the front of each nested key column title |
+---------------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
| Nested Repeats      | Key is paired to an array of objects, each of which have the same keys but with different values | Same as Nested, but since the nested keys have the same values, a number is appended in front to differentiate them (index starts at 0)     |
+---------------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+

Below shows an example of a JSON with `key1`, `key2`, and `key3` resectively representing simple, nested, and nested repeating key-value pairs.


```{json}
{
  "key1": "simple_value",
  "key2": [{
      "nested_key1": "nested_value1",
      "nested_key2": "nested_value2",
      "nested_key3": "nested_value3"
  }],
  "key3": [{
    "repeated_key1": "repeated_value1",
    "repeated_key2": "repeated_value2",
  }, {
    "repeated_key1": "repeated_value3",
    "repeated_key2": "repeated_value4"
  }, {
    "repeated_key1": "repeated_value5",
    "repeated_key2": "repeated_value6"
  }]
}
```


Below shows the resulting tabular format of the above JSON object using `fhir_pyrate`

```{}
tabular_conversion = pd.read_csv("tabular_conversion_example.csv")
tabular_conversion
```

### FHIR Example

Now that we understand the basics of how `fhir_pyrate` converts a JSON file to tabular format, let's see how an actual FHIR JSON bundle gets converted. Below is the source JSON object for the Patient query we made earlier. Compare its contents with the `patient_all` DataFrame printed above. Examples of simple, nested, and nested repeating key-value paris are respectively `id`, `telecom`, and `identifier`.

::: {.callout-caution .}
## Expand to View Raw FHIR JSON Object
:::


```{json}
{
  "resourceType": "Bundle",
  "id": "133c1f8c-bb6c-41f6-b49a-e21b24eb5a13",
  "meta": {
    "lastUpdated": "2023-04-12T15:03:02.650+00:00"
  },
  "type": "searchset",
  "total": 1,
  "link": [ {
    "relation": "self",
    "url": "https://api.logicahealth.org/FHIRResearchSynthea/open/Patient?_id=11149"
  } ],
  "entry": [ {
    "fullUrl": "https://api.logicahealth.org/FHIRResearchSynthea/open/Patient/11149",
    "resource": {
      "resourceType": "Patient",
      "id": "11149",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2023-04-06T21:10:07.000+00:00",
        "source": "#G7sxNFO6ngY1BpWr"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</a>.Version identifier: v2.4.0-404-ge7ce2295\n .   Person seed: 8026383961327351951  Population seed: 0</div>"
      },
      "extension": [ {
        "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race",
        "extension": [ {
          "url": "ombCategory",
          "valueCoding": {
            "system": "urn:oid:2.16.840.1.113883.6.238",
            "code": "1002-5",
            "display": "American Indian or Alaska Native"
          }
        }, {
          "url": "text",
          "valueString": "American Indian or Alaska Native"
        } ]
      }, {
        "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity",
        "extension": [ {
          "url": "ombCategory",
          "valueCoding": {
            "system": "urn:oid:2.16.840.1.113883.6.238",
            "code": "2186-5",
            "display": "Not Hispanic or Latino"
          }
        }, {
          "url": "text",
          "valueString": "Not Hispanic or Latino"
        } ]
      }, {
        "url": "http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName",
        "valueString": "Tia76 Fisher429"
      }, {
        "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex",
        "valueCode": "F"
      }, {
        "url": "http://hl7.org/fhir/StructureDefinition/patient-birthPlace",
        "valueAddress": {
          "city": "North Brookfield",
          "state": "Massachusetts",
          "country": "US"
        }
      }, {
        "url": "http://synthetichealth.github.io/synthea/disability-adjusted-life-years",
        "valueDecimal": 0.012998260499375002
      }, {
        "url": "http://synthetichealth.github.io/synthea/quality-adjusted-life-years",
        "valueDecimal": 30.987001739500624
      } ],
      "identifier": [ {
        "system": "https://github.com/synthetichealth/synthea",
        "value": "429ddae5-0ff8-41fc-ad45-cc7070b337aa"
      }, {
        "type": {
          "coding": [ {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "MR",
            "display": "Medical Record Number"
          } ],
          "text": "Medical Record Number"
        },
        "system": "http://hospital.smarthealthit.org",
        "value": "429ddae5-0ff8-41fc-ad45-cc7070b337aa"
      }, {
        "type": {
          "coding": [ {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "SS",
            "display": "Social Security Number"
          } ],
          "text": "Social Security Number"
        },
        "system": "http://hl7.org/fhir/sid/us-ssn",
        "value": "999-69-1055"
      }, {
        "type": {
          "coding": [ {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "DL",
            "display": "Driver's License"
          } ],
          "text": "Driver's License"
        },
        "system": "urn:oid:2.16.840.1.113883.4.3.25",
        "value": "S99938870"
      }, {
        "type": {
          "coding": [ {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "PPN",
            "display": "Passport Number"
          } ],
          "text": "Passport Number"
        },
        "system": "http://standardhealthrecord.org/fhir/StructureDefinition/passportNumber",
        "value": "X3026388X"
      } ],
      "name": [ {
        "use": "official",
        "family": "Morissette863",
        "given": [ "Alvera113" ],
        "prefix": [ "Ms." ]
      } ],
      "telecom": [ {
        "system": "phone",
        "value": "555-741-5139",
        "use": "home"
      } ],
      "gender": "female",
      "birthDate": "1987-10-01",
      "address": [ {
        "extension": [ {
          "url": "http://hl7.org/fhir/StructureDefinition/geolocation",
          "extension": [ {
            "url": "latitude",
            "valueDecimal": 42.210674475434004
          }, {
            "url": "longitude",
            "valueDecimal": -72.56410526045124
          } ]
        } ],
        "line": [ "1071 Pacocha Arcade Suite 69" ],
        "city": "West Springfield",
        "state": "Massachusetts",
        "postalCode": "01089",
        "country": "US"
      } ],
      "maritalStatus": {
        "coding": [ {
          "system": "http://terminology.hl7.org/CodeSystem/v3-MaritalStatus",
          "code": "S",
          "display": "S"
        } ],
        "text": "S"
      },
      "multipleBirthBoolean": false,
      "communication": [ {
        "language": {
          "coding": [ {
            "system": "urn:ietf:bcp:47",
            "code": "en-US",
            "display": "English"
          } ],
          "text": "English"
        }
      } ]
    },
    "search": {
      "mode": "match"
    }
  } ]
}
```


:::

Since there are a lot of columns, it might be tough to easily review them. The `info()` function from `pandas` gives us a nice view of each column title printed vertically:

```{}
patient_all.info()
```

## Processing Our DataFrame

The first step of processing a DataFrame from a FHIR query is to do an initial query for all variables and inspect the resulting dataframe. This will give us an idea of what our data looks like and what columns we actually want. We have completed this step with the previous code (that's why our `patient` variable was named `patient_all` - we pulled all variables).

After understanding what variables we actually want in our final DataFrame, our second step will be to perform a second query for the same bundle, but this include the `fhir_paths` parameter in the `steal_bundles_to_dataframe()` method to specify and rename wanted variables. Note that in oder to use this parameter, [firepath-py](https://github.com/beda-software/fhirpath-py) must be installed first.

The `fire_paths` parameter takes an array of the columns you want in your final DataFrame. We can either keep the same name as the original key, or rename it. Note that when referencing original column names, replace the underscores with periods.

Below we are creating a newly formatted DataFrame that only contains the patient ID, name, phone number, gender, and birth date.

```{}
patient = search.steal_bundles_to_dataframe(  
    resource_type="Patient",
    request_params={
        "_id": "11149"
    },
    fhir_paths=[
        "id",
        ("name", "name.0.given.0"),           # instead of name_0_given_0
        ("phone_number", "telecom.0.value"),  # instead of telecom_0_value
        "gender",
        "birthDate"
    ]
)
patient
```